In [49]:
import requests
import json
import pandas as pd
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Lines, Axis, Figure, OHLC, Tooltip

In [102]:
ticker = 'JACK'
interval = 'day'
res = requests.get('http://pipkoff-api:80/api/v0/market/candles', params={'ticker': ticker, 'interval': interval})
cont = json.loads(res.content)
candles = cont['data']['candles']
print('candles: ', len(candles))
dfc = pd.DataFrame(candles)
dfc["time"] = pd.to_datetime(dfc["time"])
dfc = dfc.set_index("time")
dfc.tail()

candles:  62


,l,o,c,v,h,interval
time,,,,,,
2021-08-09 04:00:00+00:00,99.00,104.95,99.76,126634,104.95,day
2021-08-10 04:00:00+00:00,98.16,99.00,99.27,100830,101.89,day
2021-08-11 04:00:00+00:00,100.11,100.11,101.39,113498,101.65,day
2021-08-12 04:00:00+00:00,101.00,104.50,101.76,62945,104.50,day
2021-08-13 04:00:00+00:00,101.08,101.76,102.85,98124,102.85,day


In [103]:
x_date = DateScale()
y_linear = LinearScale()
def_tt = Tooltip(
    fields=['x', 'close','open', 'high', 'low'], 
    formats = ['%Y-%m-%d %H-%M', '.2f', '.2f', '.2f', '.2f'], 
    show_labels=True)

ohlc = OHLC(x=dfc.index, y=dfc[["o","h","l","c"]],            
            scales={'x':x_date, 'y':y_linear},
            marker="candle",            
            stroke="#222", stroke_width=1.0,
            colors=["lime", "tomato"],
            tooltip=def_tt          
            )

ax_x = Axis(scale=x_date, label="Date",
            label_offset="35px", grid_color="gray",
            )
ax_y = Axis(scale=y_linear, label="Price",
            orientation="vertical", label_offset="35px",
            grid_color="gray",            
            tick_format="0.2f")

fig = Figure(marks=[ohlc],
             axes=[ax_x, ax_y],             
             title="CandleStick Chart",
             fig_margin= dict(top=60, bottom=40, left=50, right=20),
             background_style = {"fill":"#ccc"}
      )

fig.layout.height="600px"

fig


Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale()), Axis(grid_color='g…

In [161]:
%%writefile scripts/ohlc.py

import requests
import json
import pandas as pd
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Lines, Axis, Figure, OHLC, Tooltip


def show_ohlc(ticker: str, interval="day"):
    print('---hi')
    res = requests.get('http://pipkoff-api:80/api/v0/market/candles', params={'ticker': ticker, 'interval': interval})
    cont = json.loads(res.content)
    candles = cont['data']['candles']    
    dfc = pd.DataFrame(candles)
    dfc["time"] = pd.to_datetime(dfc["time"])
    dfc = dfc.set_index("time")
    
    x_date = DateScale()
    y_linear = LinearScale()
    def_tt = Tooltip(
        fields=['x', 'close','open', 'high', 'low'], 
        formats = ['%Y-%m-%d %H-%M', '.2f', '.2f', '.2f', '.2f'], 
        show_labels=True)

    ohlc = OHLC(x=dfc.index, y=dfc[["o","h","l","c"]],            
                scales={'x':x_date, 'y':y_linear},
                marker="candle",            
                stroke="#222", stroke_width=1.0,
                colors=["lime", "tomato"],
                tooltip=def_tt          
                )

    ax_x = Axis(scale=x_date, label="Date",
                label_offset="35px", grid_color="gray",
                )
    ax_y = Axis(scale=y_linear, label="Price",
                orientation="vertical", label_offset="35px",
                grid_color="gray",            
                tick_format="0.2f")

    fig = Figure(marks=[ohlc],
                 axes=[ax_x, ax_y],             
                 title="CandleStick Chart",
                 fig_margin= dict(top=60, bottom=40, left=50, right=20),
                 background_style = {"fill":"#ccc"}
          )

    fig.layout.height="600px"
    return fig


def get_intervals():
    res = requests.get('http://pipkoff-api:80/api/v0/market/intervals')
    assert res.status_code == 200
    cont = json.loads(res.content)
    return cont['data']['intervals']


def get_stock(**kwargs):
    res = requests.get('http://pipkoff-api:80/api/v0/stock', params=kwargs)
    cont = json.loads(res.content)
    return cont['data']['stocks']
    
    

Overwriting scripts/ohlc.py


In [145]:
%%bash
cat scripts/ohlc.py


import requests
import json
import pandas as pd
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Lines, Axis, Figure, OHLC, Tooltip


def show_ohlc(ticker: str, interval="day"):
    print('---hi')
    res = requests.get('http://pipkoff-api:80/api/v0/market/candles', params={'ticker': ticker, 'interval': interval})
    cont = json.loads(res.content)
    candles = cont['data']['candles']    
    dfc = pd.DataFrame(candles)
    dfc["time"] = pd.to_datetime(dfc["time"])
    dfc = dfc.set_index("time")
    
    x_date = DateScale()
    y_linear = LinearScale()
    def_tt = Tooltip(
        fields=['x', 'close','open', 'high', 'low'], 
        formats = ['%Y-%m-%d %H-%M', '.2f', '.2f', '.2f', '.2f'], 
        show_labels=True)

    ohlc = OHLC(x=dfc.index, y=dfc[["o","h","l","c"]],            
                scales={'x':x_date, 'y':y_linear},
                marker="candle",            
                stroke="#222", stroke_width=1.0,
                colors=["lim

In [155]:
%run scripts/ohlc.py

show_ohlc('TSLA', 'hour')


---hi


Figure(axes=[Axis(grid_color='gray', label='Date', label_offset='35px', scale=DateScale()), Axis(grid_color='g…